## UGCA Project 1 Report
We scraped all the comments in all the forums under the tires and wheels section of edmunds (42,890 comments)  
https://forums.edmunds.com/discussions/tagged/x/tires-wheels

## Import and clean data

In [7]:
import json
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from functools import reduce
from collections import Counter

with open("edmunds_comment_data.json", 'r') as f:
    js = json.load(f)

commentcount = 0
characterstoclean = r'?!,:,/\"-+=@#$%^&*()><{}[]|' + r"'"

commentsdict = {}
words = []
for i in js.keys():
    for j in js[i].keys():
        for k in js[i][j].keys():
            for l in js[i][j][k].keys():
                commentcount+=1
                cleanwords = []
#                 for w in word_tokenize(js[i][j][k][l]):
#                     cleanword = ''.join( c for c in w if c not in characterstoclean ).lower()
#                     cleanwords.append(cleanword)
#                     words.append(cleanword)
                comment = ''
                # to remove quoted content in comments, not 100% effective
                for s in sent_tokenize(js[i][j][k][l].lower()):
                    if "said:" not in s:
                        comment+=" " + s
                for c in characterstoclean:
                    comment = comment.replace(c,'')
                for w in word_tokenize(comment):
                    cleanwords.append(w)
                    words.append(w)                
                
                commentsdict['comment' + str(commentcount)] = ' '.join(cleanwords)
                

print("Number of comments scraped: " + str(commentcount))

Number of comments scraped: 42890


## Top 10 most frequently used words

In [8]:
cnt = Counter(words)

In [39]:
from nltk.corpus import stopwords
from nltk.corpus import brown
stopwords_set = set(stopwords.words('english'))
brown_set = set(brown.words())

In [49]:
ranked_cnt = []
for i in cnt:
    if i not in stopwords_set and i not in characterstoclean and i != '.' and i != '...' :
        ranked_cnt.append( (cnt[i], i) )
ranked_cnt.sort(reverse=True)

for i in range(len(ranked_cnt[:10])):
    print(str(i+1) + ") '" + str(ranked_cnt[i][1]) + "' appeared " + str(ranked_cnt[i][0]) + ' times')

1) 'tires' appeared 23827 times
2) 'tire' appeared 16121 times
3) 'car' appeared 9481 times
4) 'would' appeared 8370 times
5) 'new' appeared 6485 times
6) 'get' appeared 6235 times
7) 'miles' appeared 5916 times
8) 'like' appeared 5771 times
9) 'one' appeared 5637 times
10) 'good' appeared 5113 times


Note: we removed stopwords from the above list

## Top 10 Mentioned Brands

In [53]:
brands = ['BFGoodrich', 'Bridgestone', 'Continental', 'Cooper', 'Dunlop', 'Falken', 'Firestone', 'General', 'Goodyear', 'GT Radial', 'Hankook', 'Kumho', 'Michelin', 'Nexen', 'Nitto', 'Nokian', 'Pirelli', 'Sumitomo', 'Toyo', 'Uniroyal', 'Yokohama']
brand_counts = []
for i in brands:
    brand_counts.append((i, cnt[i.lower()] ))

brand_counts.sort(key=lambda x: x[1], reverse=True)

for i in range(len(brand_counts[:10])):
    print(str(i+1) + ") the brand '" + str(brand_counts[i][0]) + "' appeared " + str(brand_counts[i][1]) + ' times')

1) the brand 'Michelin' appeared 2579 times
2) the brand 'Bridgestone' appeared 1533 times
3) the brand 'Goodyear' appeared 1358 times
4) the brand 'Toyo' appeared 622 times
5) the brand 'Firestone' appeared 531 times
6) the brand 'Dunlop' appeared 499 times
7) the brand 'Continental' appeared 450 times
8) the brand 'Yokohama' appeared 441 times
9) the brand 'General' appeared 398 times
10) the brand 'Nokian' appeared 350 times


## Get all the comments that mention goodyear

In [11]:
goodyear_comments = {}
for i in commentsdict:
    if 'goodyear' in commentsdict[i]:
        goodyear_comments[i] = commentsdict[i]

In [12]:
len(goodyear_comments)

1381

## Get all the words used in comments that mention goodyear

In [13]:
words_in_goodyear_comments = []
for i in goodyear_comments:
    for j in word_tokenize(goodyear_comments[i]):
        words_in_goodyear_comments.append(j)

In [14]:
gdyr_cnt = Counter(words_in_goodyear_comments)

In [15]:
ranked_goyr_cnt = []
for i in gdyr_cnt:
    ranked_goyr_cnt.append( (gdyr_cnt[i], i) )
ranked_goyr_cnt.sort(reverse=True)
ranked_goyr_cnt[:10]

[(6110, 'the'),
 (5797, '.'),
 (3414, 'i'),
 (2700, 'and'),
 (2429, 'a'),
 (2167, 'to'),
 (2043, 'tires'),
 (1772, 'on'),
 (1390, 'of'),
 (1358, 'goodyear')]

## Comments with goodyear and ____ in them

In [16]:
comparisonstring = 'american'

comments_with_gdyr_and = []
for i in goodyear_comments:
    if comparisonstring in goodyear_comments[i]:
        comments_with_gdyr_and.append(goodyear_comments[i])
len(comments_with_gdyr_and)

8

In [26]:
comments_with_gdyr_and[0]

'hey thanks for your response . i did have them install 2 new goodyear tires to replace the 2 that i had just purchased and they are just as bad . the goodyears that i have are called american eagles . is that a private labled model maybe i would be better off going to a different walmart maybe they carry a different tire for my car . maybe they have technicians that are nice . maybe just maybe .'

Possibly filter lift value to represent associations within a maximum word count  
    Beto -- Healthcare (no more than 6 words in between)  
mds map a disimilarity matrix, not similarity matrix